# **Добавление семантики**
dataset conllu


Имеется два датасета conllu и csv, которые мне нужно соединить между собой (мне потребуются только NER и семантические классы)

Извините, я очень сильно запуталась в коде Насти, в тетрадке bertsem у нее не закончено.

Я запуталась, не могу найти ошибку, не понимаю, почему ему нужен non-fast tokenizers (e.g. instance of a `XxxTokenizerFast` class).

Правильно ли я вообще сделала?

In [47]:
import transformers
from transformers import BertModel, AutoTokenizer, BertTokenizer, PreTrainedTokenizerFast,  get_linear_schedule_with_warmup
import torch.nn.functional as F
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn, optim
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm, trange
from torch.utils.data import Dataset, DataLoader, RandomSampler, TensorDataset, SequentialSampler

In [48]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from torch.utils.data import Dataset
from torch.nn.functional import one_hot
import csv
from collections import defaultdict
import pandas as pd
import numpy as np
import re

In [49]:
# locked seed
import random
import os
import numpy as np

def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [50]:
set_all_seeds(42)

In [51]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert_model = AutoModel.from_pretrained('bert-base-cased')

У меня не совпали данные, поэтому мне пришлось вручную смотреть ошибки в своем датасете и вручную размечать. На устранение проблемы мне потребовалось около трех дней.

In [ ]:
#print(len(df_sem))  # Должны быть одинаковыми

77374


In [ ]:
#len(df_cleaned)

77315

In [52]:
# открою датасет
df_sem = pd.read_csv('semclasses.csv').fillna(method="ffill")
df_sem.head(10)

<ipython-input-52-360031de689b>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_sem = pd.read_csv('semclasses.csv').fillna(method="ffill")


,form,semclass,NER
0,Детство,TIME,O
1,провёл,TO_SPEND_TIME,O
2,в,PREPOSITION,O
3,Надьсомбате,INHABITED_LOCALITY,B-LOC
4,с,PREPOSITION,O
5,1860,CH_REFERENCE_AND_QUANTIFICATION,O
6,г,TIME,O
7,.,_,O
8,На,_,O
9,самом,_,O


Теперь все открывается правильно, дело в том, что у меня в любом случае датасеты были ассиметричны и теги съезжали рандомно на 1-4 строчки, удаление NULL тоже не помогало, все равно почему-то съезжали теги.

In [ ]:
# немного по-другому назову, чтобы адаптировать свой датасет по код Насти

In [53]:
df_sem = df_sem.rename(columns={'NER': 'tag'})
df_sem.head(2)

,form,semclass,tag
0,Детство,TIME,O
1,провёл,TO_SPEND_TIME,O


In [54]:
df_sem.head(5)

,form,semclass,tag
0,Детство,TIME,O
1,провёл,TO_SPEND_TIME,O
2,в,PREPOSITION,O
3,Надьсомбате,INHABITED_LOCALITY,B-LOC
4,с,PREPOSITION,O


In [ ]:
#df_sem.to_csv('semclasses_renamed.csv', index=False, encoding='utf-8')

In [55]:
# колонку form переименовываем в токены
if 'form' in df_sem.columns:
    df_sem = df_sem.rename(columns={'form': 'token'})

# пустая строка как словарь с теми же колонками, что и в df_sem
empty_row = {col: '' for col in df_sem.columns}
new_rows = []

# Проходим по строкам DataFrame и добавляем пустые строки после точек
for _, row in df_sem.iterrows():
    new_rows.append(row.to_dict())  #  строка DataFrame в словарь
    if row['token'] == '.' and row['tag'] == 'O':
        new_rows.append(empty_row)

new_df = pd.DataFrame(new_rows)

In [56]:
# добавила sentence_id, чтобы потом сгруппировать разделение предложений
new_df['sentence_id'] = 0
current_sentence = 1

for i in range(len(new_df)):
    if i > 0 and new_df.at[i-1, 'token'] == '.' and new_df.at[i-1, 'tag'] == 'O':
        current_sentence += 1
    new_df.at[i, 'sentence_id'] = current_sentence

In [57]:
new_df.to_csv('semclasses_with_sentences.csv', index=False, encoding='utf-8')

In [58]:
data = pd.read_csv('semclasses_with_sentences.csv').fillna(method="ffill")
data.head(10)

<ipython-input-58-2150b508546e>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = pd.read_csv('semclasses_with_sentences.csv').fillna(method="ffill")


,token,semclass,tag,sentence_id
0,Детство,TIME,O,1
1,провёл,TO_SPEND_TIME,O,1
2,в,PREPOSITION,O,1
3,Надьсомбате,INHABITED_LOCALITY,B-LOC,1
4,с,PREPOSITION,O,1
5,1860,CH_REFERENCE_AND_QUANTIFICATION,O,1
6,г,TIME,O,1
7,.,_,O,1
8,.,_,O,2
9,На,_,O,2


In [ ]:
# Я хотела разделить предложения по sentence_id, потому что они шли сплошным текстом

In [179]:
# семантический  словарь
#sem_classes = list(set(data["semclass"].values))
#sem2id = {'PAD': 0}
#for cls in sem_classes:
  #  sem2id[cls] = len(sem2id)
#id2sem = {i: cls for cls, i in sem2id.items()}

In [180]:
#tag_values = list(set(data["tag"].values))
#tag_values.append("PAD")
#tag2idx = {t: i for i, t in enumerate(tag_values)}
#tag2idx["PAD"] = -100  # Для игнорирования при loss

In [182]:
tag2idx

{'I-ORG': 1,
 'B-LOC': 2,
 'B-ORG': 3,
 'I-LOC': 4,
 'I-PER': 5,
 'B-PER': 6,
 'O': 7,
 'PAD': -100}

In [183]:
sem2id

{'PAD': 0,
 'MOTION': 1,
 'CH_EVALUATION_OF_HUMAN_TEMPER_AND_ACTIVITY': 2,
 'QUIETNESS': 3,
 'TIME': 4,
 'TO_CELEBRATE': 5,
 'TO_FABRICATE': 6,
 'CONTACT_WITH_CONTRAGENT': 7,
 'TO_SET': 8,
 'TO_BEAT_AND_PRICK': 9,
 'MANAGE_FAIL_CONDITION': 10,
 'MONEY': 11,
 'TO_PRINT_TEXT_PHOTO': 12,
 'PARTICLES': 13,
 'BEING': 14,
 'TO_PICTURE_DRAW': 15,
 'CH_INTENTION_CONCENTRATION': 16,
 'LINE_FOR_COMMUNICATION': 17,
 'TO_BE_GUIDED': 18,
 'TO_SCREEN': 19,
 'CH_SPHERE_OF_COVERAGE': 20,
 'ELECTIONS': 21,
 'COUNTRY_AS_ADMINISTRATIVE_UNIT': 22,
 'PROBLEMS_TO_SOLVE': 23,
 'TO_RATIFY': 24,
 'SPACE_BY_PARTICULAR_PROPERTIES': 25,
 'MATHEMATICAL_OBJECTS': 26,
 'BOOM': 27,
 'CLASSIFICATION_TYPES': 28,
 'RELATIVE_ENTITY': 29,
 'TO_PREVENT_SMTH': 30,
 'TO_PLAN_CREATIVE_AND_PHYSICAL_OBJECTS': 31,
 'TO_GET': 32,
 'HERITAGE': 33,
 'PRODUCT': 34,
 'POWER_RIGHT': 35,
 'ADVENTURE': 36,
 'TO_CARE_AND_BRING_UP': 37,
 'PATH_AS_DIRECTION_OF_ACTIVITY': 38,
 'PART_OF_CONSTRUCTION': 39,
 'TO_ACCUSE_AND_VINDICATE': 40,
 'CH

In [59]:
# Группировка по предложениям (что-то из кода Насти, только я поменяла на свои переменные)
class SentenceGetter:
    def __init__(self, data):
        self.data = data
        agg_func = lambda s: [(w, t, sem) for w, t, sem in zip(s["token"].values.tolist(),
                                                              s["tag"].values.tolist(),
                                                              s["semclass"].values.tolist())]
        self.grouped = self.data.groupby("sentence_id").apply(agg_func)
        self.sentences = [s for s in self.grouped]

getter = SentenceGetter(data)

<ipython-input-59-7f0afd0f5941>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.grouped = self.data.groupby("sentence_id").apply(agg_func)


In [60]:
# функция для токенизации с сохранением тегов и семклассов (у Насти было сделано что-то похожее)
def tokenize_and_preserve_labels(sentence, text_labels, sem_tags):
    tokenized_sentence = []
    labels = []
    preserved_sem_tags = []

    for word, label, sem_tag in zip(sentence, text_labels, sem_tags):
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        tokenized_sentence.extend(tokenized_word)
        labels.extend([label] * n_subwords)
        preserved_sem_tags.extend([sem_tag] * n_subwords)

    return tokenized_sentence, labels, preserved_sem_tags

In [61]:
getter = SentenceGetter(data)
MAX_LEN = 100
bs = 16

<ipython-input-59-7f0afd0f5941>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.grouped = self.data.groupby("sentence_id").apply(agg_func)


In [62]:
# здесь храняется списки предложений, тегов и семклассов
sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
labels = [[tag[1] for tag in sentence] for sentence in getter.sentences]
sem_tags = [[sem[2] for sem in sentence] for sentence in getter.sentences]

In [63]:
tag2idx = {'I-ORG': 1,
            'B-LOC': 2,
            'B-ORG': 3,
            'I-LOC': 4,
            'I-PER': 5,
            'B-PER': 6,
            'O': 7,
            'PAD': -100}

In [64]:
tag_values = list(set(data["tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [65]:
scvocab = set(data["semclass"].dropna().unique()) # здесь находятся уникальные семантические классы без пустых значений
sem2id = {'PAD': 0} # сделала словарь
for symbol in scvocab:
    sem2id[symbol] = len(sem2id)
id2sem = {i: symbol for symbol, i in sem2id.items()}

In [66]:
# токенизация
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs, sem_tag)
    for sent, labs, sem_tag in zip(sentences, labels, sem_tags)
]

In [67]:
# Подготовка данных для модели
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]
sem_tags = [token_sem_tag_pair[2] for token_sem_tag_pair in tokenized_texts_and_labels]

In [68]:
from torch.nn.utils.rnn import pad_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [69]:
# паддинг
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                    maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                    dtype="long", truncating="post")

In [28]:
class SemDataset(Dataset):
    def __init__(self, sentences, labels, sem_tags, tag2idx, sem2id, tokenizer, max_len=100):
        self.sentences = sentences # предложения
        self.labels = labels # метки NER
        self.sem_tags = sem_tags # список семантических тегов
        self.tag2idx = tag2idx # словарь меток
        self.sem2id = sem2id # словарь сем классов
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        word_labels = self.labels[idx]
        word_semtags = self.sem_tags[idx]
        # токенизация
        encoding = self.tokenizer(
            sentence,
            is_split_into_words=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            return_token_type_ids=True
        )

        word_ids = encoding.word_ids() # индексы слов для субтокенов
        label_ids = [-100] * len(word_ids) # инициализация меток (-100 = игнорировать)
        semclass_ids = [0] * len(word_ids) # bнициализация сем классов

        for i, word_idx in enumerate(word_ids):
            if word_idx is None: # токены ([CLS], [SEP], [PAD])
                continue
            semclass_ids[i] = self.sem2id[word_semtags[word_idx]] # здесь у меня сопоставляются семантические теги с ID
            if i == 0 or word_ids[i] != word_ids[i-1]:
                label_ids[i] = self.tag2idx[word_labels[word_idx]] # здесь назначаются метки

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label_ids, dtype=torch.long),
            'semclasses': torch.tensor(semclass_ids, dtype=torch.long)
        }

In [71]:
# Разделение данных (как было у Насти)
train_sent, val_sent, train_labels, val_labels, train_sem, val_sem = train_test_split(
    sentences, labels, sem_tags, test_size=0.1, random_state=4036
)

In [72]:
train_data = SemDataset(train_sent, train_labels, train_sem, tag2idx, sem2id, tokenizer, MAX_LEN)
val_data = SemDataset(val_sent, val_labels, val_sem, tag2idx, sem2id, tokenizer, MAX_LEN)

In [73]:
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, batch_size=16, sampler=train_sampler)

valid_sampler = RandomSampler(val_data)
valid_dataloader = DataLoader(val_data, batch_size=16, sampler=valid_sampler)

In [74]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [75]:
class NERModelWithSemantics(nn.Module):
    def __init__(self, bert_model, num_labels, num_sem_classes, hidden_size=768):
        super().__init__()
        self.bert = bert_model
        self.num_sem_classes = num_sem_classes

        # эмбеддинги для семантических классов
        self.semantics = nn.Embedding(num_sem_classes, 100)

        # классификатор
        self.classifier = nn.Linear(hidden_size + 100, num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids, semclasses, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True
        )

        bert_output = outputs.last_hidden_state  # получается, что здесь у нас (B, T, H) - BERT эмбеддинги
        sem_embeddings = self.semantics(semclasses)  # а здесь находятся (B, T, D) - семантические эмбеддинги

        # вот тут у меня как раз происходит конкатенация эмбеддингов
        combined = torch.cat([bert_output, sem_embeddings], dim=-1)  #  # (B, T, H+D)
        logits = self.classifier(combined)  # (B, T, num_labels)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            loss = loss_fct(logits.view(-1, logits.shape[-1]), labels.view(-1))

        return loss, logits

In [76]:
model = NERModelWithSemantics(
    bert_model=AutoModel.from_pretrained("bert-base-cased"),
    num_labels=len(tag2idx),
    num_sem_classes=len(sem2id)
)
model.to(device)

NERModelWithSemantics(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele

In [77]:
from torch.optim import AdamW
import os

In [78]:
EPOCHS = 10
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.00001) # correct_bias=False
total_steps = len(train_dataloader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [79]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [42]:
def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples): # (копия из тетрадки Насти)
    model = model.train()
    all_preds = []
    all_targets = []
    losses = []
    correct_predictions = 0

    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids=batch['token_type_ids'].to(device)
        semclasses = batch['semclasses'].to(device)
        labels = batch['labels'].to(device)

        loss, logits = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            semclasses=semclasses,
            labels=labels
        )

        preds = torch.argmax(logits, dim=2)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        all_preds.extend(preds.detach().cpu().numpy().flatten())
        all_targets.extend(labels.detach().cpu().numpy().flatten())

    # Фильтрация -100 (игнорируемые токены)
    mask = np.array(all_targets) != -100
    filtered_preds = np.array(all_preds)[mask]
    filtered_targets = np.array(all_targets)[mask]

    f1_macro = f1_score(filtered_targets, filtered_preds, average='macro')
    f1_micro = f1_score(filtered_targets, filtered_preds, average='micro')
    f1_weighted = f1_score(filtered_targets, filtered_preds, average='weighted')

    report = classification_report(filtered_targets, filtered_preds, output_dict=True)
    for class_label, metrics in report.items():
        if class_label.isdigit():
            print(f"Class {class_label}: F1-score {metrics['f1-score']:.4f}")

    return correct_predictions.double() / n_examples, np.mean(losses), f1_macro, f1_micro, f1_weighted

def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    all_preds = []
    all_targets = []
    correct_predictions = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids=batch['token_type_ids'].to(device)
            semclasses = batch['semclasses'].to(device)
            labels = batch['labels'].to(device)

            loss, logits = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                semclasses=semclasses,
                labels=labels
            )

            preds = torch.argmax(logits, dim=2)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

            all_preds.extend(preds.cpu().numpy().flatten())
            all_targets.extend(labels.cpu().numpy().flatten())

    # Фильтрация -100 (игнорируемые токены)
    mask = np.array(all_targets) != -100
    filtered_preds = np.array(all_preds)[mask]
    filtered_targets = np.array(all_targets)[mask]

    f1_macro = f1_score(filtered_targets, filtered_preds, average='macro')
    f1_micro = f1_score(filtered_targets, filtered_preds, average='micro')
    f1_weighted = f1_score(filtered_targets, filtered_preds, average='weighted')

    report = classification_report(filtered_targets, filtered_preds, output_dict=True)
    for class_label, metrics in report.items():
        if class_label.isdigit():
            print(f"Class {class_label}: F1-score {metrics['f1-score']:.4f}")

    return correct_predictions.double() / n_examples, np.mean(losses), f1_macro, f1_micro, f1_weighted

In [43]:
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss, f1_macro, f1_micro, f1_weighted = train_epoch(
        model,
        train_dataloader,
        optimizer,
        device,
        scheduler,
        len(train_data)
    )

    print(f'TRAIN:\nloss {train_loss}\naccuracy {train_acc}\nf1_macro {f1_macro}\nf1_micro {f1_micro}\nf1_weighted {f1_weighted}\n')

    val_acc, val_loss, val_f1_macro, val_f1_micro, val_f1_weighted = eval_model(
        model,
        valid_dataloader,
        device,
        len(val_data)
    )

    print(f'VAL:\nloss {val_loss}\naccuracy {val_acc}\nf1_macro {val_f1_macro}\nf1_micro {val_f1_micro}\nf1_weighted {val_f1_weighted}\n')

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

Epoch 1/10
----------


ValueError: word_ids() is not available when using non-fast tokenizers (e.g. instance of a `XxxTokenizerFast` class).